In [6]:
import requests
import json
import random
import numpy as np

from requests_oauthlib import OAuth1
from io import StringIO
from os import getenv
from time import time
from dotenv import load_dotenv

In [7]:
load_dotenv()

twitter_api_key = getenv('TWITTER_API_KEY')
twitter_api_secret = getenv('TWITTER_API_SECRET')
twitter_access_token = getenv('TWITTER_ACCESS_TOKEN')
twitter_access_secret = getenv('TWITTER_ACCESS_SECRET')

print('*Access tokens*')
print('API key:', twitter_api_key)
print('API secret:', twitter_api_secret)
print('Token:', twitter_access_token)
print('Token secret:', twitter_access_secret)

*Access tokens*
API key: 0f7uX7eXU7Vo1SQmSn45ueotS
API secret: XtSp9UEpETg93I8yRWme1UBzBxGh8QKOSxVdYgYnzWCegBuJEh
Token: 215290930-MMI0TsfZ4CQ8Y2RT6hx737MshXBlEKhxOjd8y7fv
Token secret: 125aR5ubKoVw0fUDFKQWSI6chx29as5sncHPFFaznc1pM


In [8]:
twitter_url='https://api.twitter.com/1.1/trends/place.json?id=1'

twitter_auth = OAuth1(twitter_api_key, twitter_api_secret, twitter_access_token, twitter_access_secret)

topics_mx_request = requests.get('https://trends.google.com/trends/api/topcharts?hl=en-US&tz=300&date=2017&geo=MX')
topics_global_request = requests.get('https://trends.google.com/trends/api/topcharts?hl=en-US&tz=300&date=2017&geo=GLOBAL')
cat_request = requests.get('https://gist.githubusercontent.com/Mackaber/545610649e6a32a5723af78d8069369b/raw/3a9889bf1336d9fd3e05882d7d028138e6586374/mexico_interest_dataset.json')
trends_twitter_request = requests.get(twitter_url, auth=twitter_auth)

print('*Status*')
print('Topics MX:', topics_mx_request.status_code)
print('Topics Global:', topics_global_request.status_code)
print('Topics Twitter:', trends_twitter_request.status_code)
print('Categories:', cat_request.status_code)

*Status*
Topics MX: 200
Topics Global: 200
Topics Twitter: 200
Categories: 200


In [9]:
top_mx = json.load(StringIO(topics_mx_request.text[topics_mx_request.text.find('\n')+1:]))
top_global = json.load(StringIO(topics_global_request.text[topics_global_request.text.find('\n')+1:]))
topic_twitter = trends_twitter_request.json()

categories = cat_request.json()
topics = [item for sublist in top_mx['topCharts'] for item in sublist['listItems']]       \
         + [item for sublist in top_global['topCharts'] for item in sublist['listItems']] \
         + [t['name'] for t in topic_twitter[0]['trends']]

unique_topics = set(topics)

display(len(unique_topics))

263

In [10]:
class Category:
    def __init__(self, cat_id, name, parent=None):
        self.weight = 1
        self.name = name
        self.parent = parent
        self.children = []
        self.cat_id = cat_id
        
        if parent is not None:
            parent.add_child(self)
        
    def add_child(self, child):
        self.children.append(child)
        child.parent = self
        
        for c in children:
            c.weight = 1 / len(children)
            
    def add_parent(self, parent):
        parent.add_child(self)
        
    def is_descendant_of(self, cat):
        if(self.parent is None):
            return False
        
        if(self.parent is cat):
            return True
        
        return self.parent.is_descendant_of(cat)
        
class Topic:
    def __init__(self, name, categories=[]):
        self.name = name
        self.categories = categories[:]
        
        # Remover ascendencia de categorias (son redundantes)
        # Complejidad: O(mn^2) donde m es la altura maxima del arbol de categorias.
        # Quizas se pueda mejorar pero dudo que un tema tenga muchas categorias
        for c1 in self.categories:
            for c2 in self.categories:
                if c1.is_descendant_of(c2):
                    self.categories.remove(c1)
    
    
    '''
        Esta funcion retorna un vector con los pesos de categoria calculados de la
        siguiunte forma:
        
        - Si la categoria fue asignada al tema, esta tiene un peso de 1
        - Si la categoria es un hijo de una categoria asignada, se pondera
          esta categoria proporcional a la cantidad de hermanos que tiene
        - Si la categoria es padre de una categoria asignada, se pondera esta
          categoria proporcional a la ponderacion del hijo 
    '''
    def categorization_array(self, n_cat):
        mat = np.zeros(n_cat)
        
        for cat in self.categories:
            mat[cat.cat_id] = 1
            
            for ch in cat.children:
                # Cada hijo tiene un peso equivalente a
                # una fraccion del padre
                mat[ch.cat_id] = ch.weight
            
            # Iterar por padres
            par = cat.parent
            w = cat.weight
            while(par is not None):
                # Cada padre tiene un peso equivalente
                # a la fraccion del hijo
                mat[par.cat_id] = w
                w *= par.weight
        
        return mat
                
                